In [3]:
from data_loader import ImgDataset,TgtImgDataset, VideoDataset
from sampler import RandomIdentityBatchSampler,RandomTrackletBatchSampler,RandomIdentitySampler
from data_manager import Mars,DukeMTMC

import torch
from torchvision import transforms as T
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from model import FeatureExtractor,SrcReidModel,TgtPartModel,ModifiedTripletLoss,TripletLoss
from torch import optim
from torch.optim import lr_scheduler

In [55]:
def extract_feature(dataloader, feature_extractor):
    features = []
    pids = []
    camids = []
    tracklets = []
    paths = []
    with torch.no_grad():
        for batch_idx,batch in enumerate(dataloader):
            imgs = batch[0].cuda(3)
            pid = batch[1]
            camid = batch[2]
            path = list(batch[3])
            if len(batch) > 4:
                tracklet = batch[4]
                tracklets.append(tracklet)

            feature = feature_extractor(imgs)

            features += [feature[:,:,0].cpu()] 
            pids += [pid]
            camids += [camid]
            paths += path
            print(batch_idx)
        features = torch.cat(features,dim=0)
        pids = torch.cat(pids,dim=0).numpy()
        if len(tracklets) > 0:
            tracklets = torch.cat(tracklets,dim=0).numpy()
    return features,pids,camids,paths,tracklets
    

In [ ]:
def compute_distmat(a,b):
    m = a.size(0)
    n = b.size(0)
    distmat = torch.pow(a,2).sum(dim=1,keepdim=True).expand(m,n) + \
                         torch.pow(b,2).sum(dim=1,keepdim=True).expand(n,m).t()
    distmat.addmm_(1,-2,a,b.t())
    distmat = distmat.clamp(min=1e-12).sqrt() 
    return distmat.numpy()

In [ ]:
class CrossEntropyLabelSmooth(nn.Module):
    """Cross entropy loss with label smoothing regularizer.

    Reference:
    Szegedy et al. Rethinking the Inception Architecture for Computer Vision. CVPR 2016.
    Equation: y = (1 - epsilon) * y + epsilon / K.

    Args:
        num_classes (int): number of classes.
        epsilon (float): weight.
    """
    def __init__(self, num_classes, epsilon=0.1, use_gpu=True):
        super(CrossEntropyLabelSmooth, self).__init__()
        self.num_classes = num_classes
        self.epsilon = epsilon
        self.use_gpu = use_gpu
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, inputs, targets):
        """
        Args:
            inputs: prediction matrix (before softmax) with shape (batch_size, num_classes)
            targets: ground truth labels with shape (num_classes)
        """
        log_probs = self.logsoftmax(inputs)
        targets = torch.zeros(log_probs.size()).scatter_(1, targets.unsqueeze(1).data.cpu(), 1)
        if self.use_gpu: targets = targets.cuda()
        targets = Variable(targets, requires_grad=False)
        targets = (1 - self.epsilon) * targets + self.epsilon / self.num_classes
        loss = (- targets * log_probs).mean(0).sum()
        return loss

In [13]:

class GlobalVar(object):
    def __init__(self):

        self._tgt_batch_size = 8
        self.seq_len = 4
        self.batch_size_sum = 64
        self.num_parts = 6
        self.gpu_ids = [3]
        self.feature_extractor = FeatureExtractor(self.num_parts)
        if len(self.gpu_ids > 0):
            self.use_gpu = True
            torch.cuda.set_device(self.gpu_ids)
            self.feature_extractor = nn.DataParallel(self.feature_extractor,self.gpu_ids)
        else:
            self.use_gpu = False

        self.save_path = './Logs'
        self.src_data = DukeMTMC()
        self.tgt_data = Mars()

        self.train_transfrom = T.Compose(
            [
                T.Resize((256,128)),
                T.RandomHorizontalFlip(),
                T.ToTensor(),
                T.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
            ]
        )

        self.test_transform = T.Compose(
            [
                T.Resize((256,128)),
                T.ToTensor(),
                T.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
            ]
        )
        



    @property
    def tgt_batch_size(self):
        return self._tgt_batch_size

    @tgt_batch_size.setter
    def tgt_batch_size(self, value):

        if value*self.seq_len > self.batch_size_sum:
            self._tgt_batch_size = (
                self.batch_size_sum // self.seq_len)*self.seq_len
        elif value < 0:
            self._tgt_batch_size = 0
        else:
            self._tgt_batch_size = value

    @property
    def src_batch_size(self):
        return self.batch_size_sum - self._tgt_batch_size*self.seq_len

    def step(self):
        pass

    def set_global_var(self,name,value):
        setattr(self,name,value)

In [14]:
glob_var = GlobalVar()

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value.
       
       Code imported from https://github.com/pytorch/examples/blob/master/imagenet/main.py#L247-L262
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        

In [1]:
!nvidia-smi

Sun Mar 29 08:13:51 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.130                Driver Version: 384.130                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:02:00.0 Off |                  N/A |
| 78%   87C    P2   223W / 250W |   8365MiB / 11172MiB |     79%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:03:00.0 Off |                  N/A |
| 68%   83C    P2    83W / 250W |   8961MiB / 11172MiB |     99%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [4]:
tgt_labels = [torch.IntTensor([i]*64) for i in range(6)]

In [31]:
for i in range(len(tgt_labels)):
    if True:
        tgt_labels[i] = tgt_labels[i].cuda()
    tgt_labels[i][:,i] = 1

In [5]:
a = torch.tensor()

[tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.int32), tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=torch.int32), tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=torch.int32), tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4

In [ ]:
def TrainStage2(object):
    def __init__(self,glob_var):
        self.glob_var = glob_var
    def get_dataloader(self,)

In [6]:
glob_var = GlobalVar()

In [16]:
train_stage1 = TrainStage1(glob_var)

In [17]:
train_stage1.train()

torch.Size([8, 2048])
torch.Size([8, 8])
tensor(66.5332)


UnboundLocalError: local variable 'imgs' referenced before assignment

In [19]:
import numpy as np
a = np.array([3.6,4.2]).astype('int')
print(a)

[3 4]


In [ ]:
from utils  import GlobalVar

class TrainStage2(object):
    def __init__(self.glob_var):

    def get_dataloader()